## Test Gemini Pro api calls

In [1]:
from make_gemini_api_call import gemini_api_call

/Users/Deependu/constitution-gpt/backend/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
context = "My name is Deependu, and I love pizza."
question = "what does Deependu loves?"
gemini_response = gemini_api_call(context=context, question=question)
gemini_response

'pizza'

---

## Test langchain PyPdf for splitting pdf into pages

In [3]:
from pdf_splitter import split_pdf_into_pages, split_pages_into_text

In [4]:
pages = split_pdf_into_pages(pdf_path="../assets/IndianConstitution.pdf")
len(pages), pages[0], dict(pages[0]).keys(), dict(pages[0])['metadata']

(403,
 Document(page_content='PREFACE \n \nThis is the  fifth  pocket size edition of the Constitution of \nIndia in the diglot form. In this edition, the text of the \nConstitution of India has been brought up-to-date by \nincorporating therein all the amendments up to the Constitution \n(One Hundred and Fifth Amendment) Act, 2021. The foot notes \nbelow the text indicate the Constitution Amendment Acts by \nwhich such amendments have been made.  \nThe Constitution (One Hundredth Amendment) Act, 2015 \ncontaining details of acquired and transferred territories \nbetween the Governments of India and Bangladesh has been \nprovided in APPENDIX I. \nThe Constitution (Application to Jammu and Kashmir) \nOrder, 2019 and the declaration under article 370(3) of the \nConstitution have been provided respectively in Appendix II and \nAppendix III for reference. \n \n \nNew Delhi;                                              Dr. Reeta Vasishta, \n                                 Secretary to the

In [5]:
page_0_content = dict(pages[0])['page_content']

len(page_0_content)

971

---

## Test hugging face `text to embedding`

- accepts maximum sequence length 384
- gives embedding dimension of 768

In [6]:
texts = split_pages_into_text(pages, chunk_size=600, chunk_overlap=100)

print(len(texts))

1285


In [7]:
my_first_txt = dict(texts[0])['page_content']
my_second_txt = dict(texts[1])['page_content']

print(f"{len(my_first_txt)=}; {my_first_txt=}")
print(f"="*80)
print(f"{len(my_second_txt)=}; {my_second_txt=}")

len(my_first_txt)=971; my_first_txt='PREFACE \n \nThis is the  fifth  pocket size edition of the Constitution of \nIndia in the diglot form. In this edition, the text of the \nConstitution of India has been brought up-to-date by \nincorporating therein all the amendments up to the Constitution \n(One Hundred and Fifth Amendment) Act, 2021. The foot notes \nbelow the text indicate the Constitution Amendment Acts by \nwhich such amendments have been made.  \nThe Constitution (One Hundredth Amendment) Act, 2015 \ncontaining details of acquired and transferred territories \nbetween the Governments of India and Bangladesh has been \nprovided in APPENDIX I. \nThe Constitution (Application to Jammu and Kashmir) \nOrder, 2019 and the declaration under article 370(3) of the \nConstitution have been provided respectively in Appendix II and \nAppendix III for reference. \n \n \nNew Delhi;                                              Dr. Reeta Vasishta, \n                                 Secretary

In [8]:
dict(texts[0])

{'page_content': 'PREFACE \n \nThis is the  fifth  pocket size edition of the Constitution of \nIndia in the diglot form. In this edition, the text of the \nConstitution of India has been brought up-to-date by \nincorporating therein all the amendments up to the Constitution \n(One Hundred and Fifth Amendment) Act, 2021. The foot notes \nbelow the text indicate the Constitution Amendment Acts by \nwhich such amendments have been made.  \nThe Constitution (One Hundredth Amendment) Act, 2015 \ncontaining details of acquired and transferred territories \nbetween the Governments of India and Bangladesh has been \nprovided in APPENDIX I. \nThe Constitution (Application to Jammu and Kashmir) \nOrder, 2019 and the declaration under article 370(3) of the \nConstitution have been provided respectively in Appendix II and \nAppendix III for reference. \n \n \nNew Delhi;                                              Dr. Reeta Vasishta, \n                                 Secretary to the Government 

---

## Test Qdrant vectors

In [3]:
from hugging_face_text_to_embedding import get_embedding_length, text_to_embeddings
from qdrant_vector_store import create_qdrant_collection, upload_to_qdrant_collection, search_qdrant_collection

In [4]:
collection_name="constitution-gpt-embeddings-collection"
embedding_size = get_embedding_length()

create_qdrant_collection(collection_name=collection_name, embedding_size=embedding_size)

Collection constitution-gpt-embeddings-collection already exists.


In [11]:
my_ids = []
my_vectors = []
my_payloads = []

for idx, txt in enumerate(texts,1):
    my_ids.append(idx)
    _content = dict(txt)["page_content"]
    curr_emb = text_to_embeddings(_content)
    my_vectors.append(curr_emb)
    my_payloads.append({"content":_content})

my_ids[:5],my_payloads[:1], my_vectors[0][:5]

([1, 2, 3, 4, 5],
 [{'content': 'PREFACE \n \nThis is the  fifth  pocket size edition of the Constitution of \nIndia in the diglot form. In this edition, the text of the \nConstitution of India has been brought up-to-date by \nincorporating therein all the amendments up to the Constitution \n(One Hundred and Fifth Amendment) Act, 2021. The foot notes \nbelow the text indicate the Constitution Amendment Acts by \nwhich such amendments have been made.  \nThe Constitution (One Hundredth Amendment) Act, 2015 \ncontaining details of acquired and transferred territories \nbetween the Governments of India and Bangladesh has been \nprovided in APPENDIX I. \nThe Constitution (Application to Jammu and Kashmir) \nOrder, 2019 and the declaration under article 370(3) of the \nConstitution have been provided respectively in Appendix II and \nAppendix III for reference. \n \n \nNew Delhi;                                              Dr. Reeta Vasishta, \n                                 Secretary to 

In [12]:
# let's send 100 items at once, and not all at once
total_item_at_once = 10
for i in range(0, len(my_payloads), total_item_at_once):
    upload_to_qdrant_collection(
        collection_name=collection_name,
        ids=my_ids[i : i + total_item_at_once],
        payloads=my_payloads[i : i + total_item_at_once],
        vectors=my_vectors[i : i + total_item_at_once],
    )

print("all done ✅")

operation_info=UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=8, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=9, status=<UpdateStatus.COMPLETED: 'completed'>)
operation_info=UpdateResult(operation_id=10, status=<UpdateStatus.COMPLETED: 'completed'>)
operation

---

## Search in the Qdrant database

In [1]:
from hugging_face_text_to_embedding import text_to_embeddings

my_question = "What criteria should I fulfill to be considered a citizen of India?"

my_question_embedding = text_to_embeddings(my_text=my_question)

type(my_question_embedding), len(my_question_embedding)

/Users/Deependu/constitution-gpt/backend/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


(list, 768)

In [5]:
search_result = search_qdrant_collection(collection_name=collection_name, query_vector=my_question_embedding, limit=5)
search_result[:1]

[ScoredPoint(id=70, version=6, score=0.6367292, payload={'content': 'defined in the Government of India Act, 1935 (as originally enacted), and who \nis ordinarily residing in any country outside India as so defined shall be deemed \nto be a citizen of India if he has been registered as a citizen of India by the \ndiplomatic or consular representative of India in the country where he is for the \ntime being residing on an application made by him therefor to such diplomatic \nor consular representative, whether before or after the commencement of this \nConstitution, in the form and manner prescribed by the Government of the \nDominion of India or the Government of India. \n9. Persons voluntarily acquiring citizenship of a foreign State not to \nbe citizens. — No person shall be a citizen of India by virtue of article 5, or be \ndeemed to be a citizen of India by virtue of article 6 or article 8, if he has \nvoluntarily acquired the citizenship of any foreign State. \n10. Continuance of 

In [6]:
context = ""

for res in search_result:
    _con = dict(res)["payload"]["content"]
    context = context + _con

print(f"{context=}")

context='defined in the Government of India Act, 1935 (as originally enacted), and who \nis ordinarily residing in any country outside India as so defined shall be deemed \nto be a citizen of India if he has been registered as a citizen of India by the \ndiplomatic or consular representative of India in the country where he is for the \ntime being residing on an application made by him therefor to such diplomatic \nor consular representative, whether before or after the commencement of this \nConstitution, in the form and manner prescribed by the Government of the \nDominion of India or the Government of India. \n9. Persons voluntarily acquiring citizenship of a foreign State not to \nbe citizens. — No person shall be a citizen of India by virtue of article 5, or be \ndeemed to be a citizen of India by virtue of article 6 or article 8, if he has \nvoluntarily acquired the citizenship of any foreign State. \n10. Continuance of the rights of citizenship. —Every person who is orbe citizen

---

## langchain template for making llm api call

In [7]:
new_prompt = f"""Use the following pieces of context to answer the question at the end. If the answer is not present in the context, please answer with "I am sorry, but I couldn't find the answer".

context: `{context}`

Question: `{my_question}`
Helpful Answer:"""

print(new_prompt)

Use the following pieces of context to answer the question at the end. If the answer is not present in the context, please answer with "I am sorry, but I couldn't find the answer".

context: `defined in the Government of India Act, 1935 (as originally enacted), and who 
is ordinarily residing in any country outside India as so defined shall be deemed 
to be a citizen of India if he has been registered as a citizen of India by the 
diplomatic or consular representative of India in the country where he is for the 
time being residing on an application made by him therefor to such diplomatic 
or consular representative, whether before or after the commencement of this 
Constitution, in the form and manner prescribed by the Government of the 
Dominion of India or the Government of India. 
9. Persons voluntarily acquiring citizenship of a foreign State not to 
be citizens. — No person shall be a citizen of India by virtue of article 5, or be 
deemed to be a citizen of India by virtue of art

---

## Make Gemini api call

In [8]:
from make_gemini_api_call import gemini_api_call

In [10]:
my_rag_output = gemini_api_call(context=context, question=my_question)
my_rag_output

'To be considered a citizen of India, you must meet one of the following criteria:\n\n1. You were born in India.\n2. At least one of your parents was born in India.\n3. You have been ordinarily resident in India for at least five years immediately preceding the commencement of the Constitution.\n4. You have migrated to India from Pakistan and meet certain conditions, such as being born in India or having a parent or grandparent who was born in India.\n5. You have been registered as a citizen of India by an Indian diplomatic or consular representative in the country where you are residing.\n6. You have not voluntarily acquired the citizenship of any foreign state.'

In [11]:
print(my_rag_output)

To be considered a citizen of India, you must meet one of the following criteria:

1. You were born in India.
2. At least one of your parents was born in India.
3. You have been ordinarily resident in India for at least five years immediately preceding the commencement of the Constitution.
4. You have migrated to India from Pakistan and meet certain conditions, such as being born in India or having a parent or grandparent who was born in India.
5. You have been registered as a citizen of India by an Indian diplomatic or consular representative in the country where you are residing.
6. You have not voluntarily acquired the citizenship of any foreign state.
